https://www.kaggle.com/datasets/vikrishnan/boston-house-prices

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor

In [4]:
from plots.error_line import error_line
from plots.error_scatter import error_scatter

### Data Preparation

In [5]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
df = pd.read_csv("F:/Data/datas/housing.csv", delimiter=r"\s+", names=column_names)

In [6]:
CAT_COLS = ['CHAS', 'RAD']
NUM_COLS = df.columns[~df.columns.isin(CAT_COLS)].to_list()
NUM_COLS.remove('MEDV')

In [7]:
df.isna().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

### Data Preprocessing

In [8]:
print("skewness")
print('yeo-johnson: ', pd.DataFrame(PowerTransformer(method='yeo-johnson').fit_transform(df[NUM_COLS])).skew().abs().sum())
print('sqrt: ', np.sqrt(df[NUM_COLS]).skew().abs().sum())
print('cbrt: ', np.cbrt(df[NUM_COLS]).skew().abs().sum())
print('log: ', np.log(df[NUM_COLS]).skew().abs().sum())
print('log1p: ', np.log1p(df[NUM_COLS]).skew().abs().sum())

skewness
yeo-johnson:  4.604747305574276
sqrt:  10.816156352028656
cbrt:  10.102657627038495
log:  10.0233009903071
log1p:  11.56286085253241


F:\Python\tester\lib\site-packages\pandas\core\internals\blocks.py:366: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
F:\Python\tester\lib\site-packages\pandas\core\nanops.py:1248: RuntimeWarning: invalid value encountered in subtract
  adjusted = values - mean


Since the yeo-johnson technique results in the lowest skew value, we will further use the "yeo-johnson" transformation technique in our model pipeline.

### Predictive Modeling using Sklearn

In [9]:
class WinsorizingOutlier(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        self.winsorizing_outliers = {}

    def fit(self, X: pd.DataFrame, y=None):        
        self.winsorizing_outliers = self.get_outliers_caps(X)
        
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:    
        X_transformed = X.copy()
        
        for col in self.columns:
            top_cap = self.winsorizing_outliers[col]['top_cap']
            bottom_cap = self.winsorizing_outliers[col]['bottom_cap']
            X_transformed[col] = X_transformed[col].clip(lower=bottom_cap, upper=top_cap)
        
        return X_transformed

    def get_outliers_caps(self, df: pd.DataFrame) -> dict:
        outliers = {}
        for col in self.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1

            top_cap = Q3 + (IQR * 1.5)
            bottom_cap = Q1 - (IQR * 1.5)
            
            outliers[col] = {"top_cap": top_cap, "bottom_cap": bottom_cap}
        
        return outliers

In [10]:
target_caps = WinsorizingOutlier(['MEDV']).get_outliers_caps(df)['MEDV']

In [11]:
X = df.drop(['MEDV'], axis = 1)
y = df['MEDV']

y_transformed = df['MEDV'].clip(lower=target_caps['top_cap'], upper=target_caps['bottom_cap'])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.3, random_state=42)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [13]:
preprocessed = [
    ('outlier_imputer', WinsorizingOutlier(columns=NUM_COLS)),
    ('power_transformer', PowerTransformer(method='yeo-johnson')), 
    ('scaler', StandardScaler()),    
]

#### Linear Regression

In [14]:
pipeline_lr = Pipeline(preprocessed + [('regressor', LinearRegression())])
pipeline_lr.fit(X_train, y_train)

Pipeline(steps=[('outlier_imputer',
                 WinsorizingOutlier(columns=['CRIM', 'ZN', 'INDUS', 'NOX', 'RM',
                                             'AGE', 'DIS', 'TAX', 'PTRATIO',
                                             'B', 'LSTAT'])),
                ('power_transformer', PowerTransformer()),
                ('scaler', StandardScaler()),
                ('regressor', LinearRegression())])

In [15]:
cv_scores = cross_val_score(pipeline_lr, X_train, y_train, cv=kf, scoring="r2")
test_score = pipeline_lr.score(X_test, y_test)
print(f"r2-cv: {cv_scores.mean()}")
print(f"r2-test: {test_score.mean()}")

r2-cv: 0.7733018589121157
r2-test: 0.7870359181579546


#### SVR

In [16]:
pipeline_svr = Pipeline(preprocessed + [('regressor', SVR())])
pipeline_svr.fit(X_train, y_train)

Pipeline(steps=[('outlier_imputer',
                 WinsorizingOutlier(columns=['CRIM', 'ZN', 'INDUS', 'NOX', 'RM',
                                             'AGE', 'DIS', 'TAX', 'PTRATIO',
                                             'B', 'LSTAT'])),
                ('power_transformer', PowerTransformer()),
                ('scaler', StandardScaler()), ('regressor', SVR())])

In [17]:
cv_scores = cross_val_score(pipeline_svr, X_train, y_train, cv=kf, scoring="r2")
test_score = pipeline_svr.score(X_test, y_test)
print(f"r2-cv: {cv_scores.mean()}")
print(f"r2-test: {test_score.mean()}")

r2-cv: 0.7520452186347681
r2-test: 0.7748363302115993


#### XGBRegressor

In [18]:
pipeline_xgb = Pipeline(preprocessed + [('regressor', XGBRegressor())])
pipeline_xgb.fit(X_train, y_train)

Pipeline(steps=[('outlier_imputer',
                 WinsorizingOutlier(columns=['CRIM', 'ZN', 'INDUS', 'NOX', 'RM',
                                             'AGE', 'DIS', 'TAX', 'PTRATIO',
                                             'B', 'LSTAT'])),
                ('power_transformer', PowerTransformer()),
                ('scaler', StandardScaler()),
                ('regressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [19]:
cv_scores = cross_val_score(pipeline_xgb, X_train, y_train, cv=kf, scoring="r2")
test_score = pipeline_xgb.score(X_test, y_test)
print(f"r2-cv: {cv_scores.mean()}")
print(f"r2-test: {test_score.mean()}")

r2-cv: 0.8341838171285738
r2-test: 0.8519529471138473


### Model Comparisons using PyCaret

In [20]:
from pycaret.regression import *

In [21]:
regression_setup = setup(
    data=X_train, 
    target=y_train, 
    train_size=0.7, 
    remove_outliers=True, 
    transformation=True,
    normalize=False,  
    fold=5, 
    session_id=42
)

,Description,Value
0,Session id,42
1,Target,MEDV
2,Target type,Regression
3,Original data shape,"(354, 14)"
4,Transformed data shape,"(341, 14)"
5,Transformed train set shape,"(234, 14)"
6,Transformed test set shape,"(107, 14)"
7,Numeric features,13
8,Preprocess,True
9,Imputation type,simple


In [22]:
X_test_transformed, y_test_transformed = regression_setup.pipeline.transform(X_test, y_test)

In [23]:
compare_models(sort='R2', budget_time=1)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,1.8927,6.9431,2.6287,0.8740,0.1247,0.0971,0.1940
catboost,CatBoost Regressor,1.9880,7.6957,2.7641,0.8636,0.1349,0.1051,1.7120
et,Extra Trees Regressor,2.0567,8.2296,2.8359,0.8558,0.1319,0.1064,0.2360
lightgbm,Light Gradient Boosting Machine,2.1021,8.2645,2.8578,0.8529,0.1328,0.1091,0.2100
rf,Random Forest Regressor,2.1433,8.4270,2.8857,0.8486,0.1409,0.1150,0.2960
xgboost,Extreme Gradient Boosting,2.2072,9.0698,2.9845,0.8403,0.1412,0.1146,0.1980
ada,AdaBoost Regressor,2.3220,9.0268,2.9959,0.8392,0.1416,0.1204,0.2140
lar,Least Angle Regression,2.7002,11.5606,3.3830,0.7930,0.1863,0.1456,0.1580
ridge,Ridge Regression,2.8727,12.1575,3.4789,0.7822,0.1706,0.1489,0.1480
dt,Decision Tree Regressor,2.8050,14.8933,3.8457,0.7330,0.1805,0.1471,0.1560


GradientBoostingRegressor(random_state=42)

### Model Testing and Validation

#### Gradient Boosting Regression

In [24]:
gbr_model = create_model('gbr')
gbr_model = tune_model(gbr_model, optimize="R2")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.9009,7.6503,2.7659,0.8706,0.1141,0.0853
1,2.1258,7.7753,2.7884,0.8789,0.1733,0.1399
2,1.8826,6.3779,2.5254,0.8983,0.1141,0.0873
3,1.6861,5.3743,2.3183,0.9037,0.1105,0.0862
4,1.8680,7.5376,2.7455,0.8184,0.1117,0.0870
Mean,1.8927,6.9431,2.6287,0.8740,0.1247,0.0971
Std,0.1399,0.9302,0.1817,0.0303,0.0243,0.0214


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.9034,10.7503,3.2788,0.8182,0.1246,0.0809
1,2.1755,7.6955,2.7741,0.8802,0.1734,0.1457
2,2.0070,7.2373,2.6902,0.8846,0.1146,0.0905
3,1.7655,5.6842,2.3842,0.8981,0.1217,0.0955
4,1.5962,5.2891,2.2998,0.8726,0.1028,0.0796
Mean,1.8895,7.3313,2.6854,0.8707,0.1274,0.0985
Std,0.1986,1.9347,0.3462,0.0275,0.0242,0.0244


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [25]:
test_score = gbr_model.score(get_config('X_test_transformed'), get_config('y_test_transformed'))
validation_score = gbr_model.score(X_test_transformed, y_test_transformed)
print("test score r2:", test_score)
print("validation score r2:", validation_score)

test score r2: 0.8271024540591156
validation score r2: 0.7940454312678453


#### XGBoost

In [26]:
xgb_model = create_model('xgboost')
xgb_model = tune_model(xgb_model, optimize="R2")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.9810,8.0197,2.8319,0.8644,0.1233,0.0918
1,2.8149,13.9915,3.7405,0.7821,0.2110,0.1804
2,2.4267,9.1917,3.0318,0.8534,0.1355,0.1169
3,1.9980,6.8500,2.6173,0.8772,0.1216,0.0989
4,1.8155,7.2962,2.7012,0.8242,0.1146,0.0849
Mean,2.2072,9.0698,2.9845,0.8403,0.1412,0.1146
Std,0.3649,2.5851,0.4030,0.0339,0.0356,0.0346


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.0022,8.8593,2.9765,0.8502,0.1206,0.0881
1,2.2596,8.3685,2.8928,0.8697,0.1856,0.1559
2,2.3456,9.3289,3.0543,0.8513,0.1277,0.1068
3,2.2026,8.3765,2.8942,0.8499,0.1487,0.1211
4,1.8517,6.3796,2.5258,0.8463,0.1230,0.0961
Mean,2.1324,8.2626,2.8687,0.8535,0.1411,0.1136
Std,0.1802,1.0063,0.1816,0.0083,0.0243,0.0239


Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [27]:
test_score = xgb_model.score(get_config('X_test_transformed'), get_config('y_test_transformed'))
validation_score = xgb_model.score(X_test_transformed, y_test_transformed)
print("test score r2:", test_score)
print("validation score r2:", validation_score)

test score r2: 0.782863421018545
validation score r2: 0.8184502079149905


#### Catboost

In [28]:
catboost_model = create_model('catboost')
catboost_model = tune_model(catboost_model, optimize="R2")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.9538,9.6064,3.0994,0.8376,0.1211,0.0849
1,2.3523,8.6878,2.9475,0.8647,0.1854,0.1578
2,2.0775,7.5772,2.7527,0.8792,0.1256,0.0987
3,1.9103,6.4998,2.5495,0.8835,0.1286,0.1003
4,1.6464,6.1076,2.4714,0.8529,0.1136,0.0836
Mean,1.9880,7.6957,2.7641,0.8636,0.1349,0.1051
Std,0.2300,1.3116,0.2358,0.0169,0.0258,0.0272


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.1853,9.5254,3.0863,0.8389,0.1412,0.0973
1,2.2080,7.8028,2.7933,0.8785,0.1750,0.1486
2,2.4153,10.1643,3.1882,0.8379,0.1551,0.1209
3,2.2003,7.4692,2.7330,0.8661,0.1343,0.1147
4,2.2646,8.3362,2.8872,0.7992,0.1327,0.1115
Mean,2.2547,8.6596,2.9376,0.8441,0.1476,0.1186
Std,0.0847,1.0266,0.1733,0.0274,0.0158,0.0169


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [71]:
test_score = catboost_model.score(get_config('X_test_transformed'), get_config('y_test_transformed'))
validation_score = catboost_model.score(X_test_transformed, y_test_transformed)
print("data-test r2:", test_score)
print("data-unseen r2:", validation_score)

data-test r2: 0.8491550291630193
data-unseen r2: 0.8288589952599161


In [61]:
y_pred_test = catboost_model.predict(get_config('X_test_transformed'))
y_pred_unseen = catboost_model.predict(X_test_transformed)

In [70]:
mae_test = mean_absolute_error(get_config('y_test_transformed'), y_pred_test)
mae_unseen = mean_absolute_error(y_test_transformed, y_pred_unseen)
print("data-test mean absolute error:", mae_test)
print("data-unseen mean absolute error:", mae_unseen)

data-test mean absolute error: 2.12846779049372
data-unseen mean absolute error: 1.9143312380010808


### Catboost Model Evaluation (The best model)

In [30]:
predictions = predict_model(catboost_model)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,2.1285,9.2472,3.0409,0.8492,0.1598,0.1178


In [46]:
fig_error = error_scatter(predictions['MEDV'], predictions['prediction_label'])

In [56]:
fig_error

In [57]:
evaluate_model(catboost_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…